# Session 3 - Part 1
This interactive notebook demonstrates how to predict retinotopic maps from anatomical data using a geometric deep learning model of retinotopy. For more details about the model, please check [here](https://www.sciencedirect.com/science/article/pii/S1053811921008971).

**Requirements:**
- freesurfer directory;
- HCP "fs_LR-deformed_to-fsaverage" surfaces, available [here]( https://github.com/Washington-University/HCPpipelines/tree/master/global/templates/standard_mesh_atlases/resample_fsaverage).

## Requirement 1: Download HCP template surfaces
Create a "templates" (for example, /home/jovyan/templates/) folder and download the following files from the link above:
1. "fs_LR-deformed_to-fsaverage.R.sphere.32k_fs_LR.surf.gii";
2. "fs_LR-deformed_to-fsaverage.L.sphere.32k_fs_LR.surf.gii";


## Requirement 2: FreeSurfer directory
Ensure your FreeSurfer directory is organized as follows:
```
.
├── 102311
│   └── surf
│       ├── lh.pial
│       ├── lh.sphere
│       ├── lh.sphere.reg
│       ├── lh.thickness
│       ├── lh.white
│       ├── rh.graymid
│       ├── rh.graymid.H
│       ├── rh.graymid.K
│       ├── rh.pial
│       ├── rh.sphere
│       ├── rh.sphere.reg
│       ├── rh.thickness
│       └── rh.white
├── 102816
│   └── surf
│       └── ...
```

We use the midthickness surface instead of the white or pial surfaces in the current prediction pipeline. Given that generating the midthickness surface with FreeSurfer takes up to 2 hours, we provide the midthickness surfaces to speed up retinotopic map generation.

## Creating a conda environment for data visualization
You can run the following cell or create the conda environment via a terminal window.

In [ ]:
%%bash
conda create -n deepretinotopy_2 python=3.8 ipykernel
conda activate deepretinotopy

pip install -r requirements.txt;

## Setting up Neurodesk

In [2]:
%%capture
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  os.environ["LD_PRELOAD"] = "";
  os.environ["APPTAINER_BINDPATH"] = "/content,/tmp,/cvmfs"
  os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
  os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

  !curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
  !chmod +x googlecolab_setup.sh
  !./googlecolab_setup.sh

  os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

In [3]:
# Output CPU information:
!cat /proc/cpuinfo | grep 'vendor' | uniq
!cat /proc/cpuinfo | grep 'model name' | uniq

vendor_id	: AuthenticAMD
model name	: AMD EPYC-Rome Processor


## Predicting retinotopy maps with deepRetinotopy

In [4]:
# load deepretinotopy
import lmod
import os
await lmod.load('deepretinotopy/1.0.2')
await lmod.list()

['deepretinotopy/1.0.2']

In [8]:
# generate polar angle maps
%%bash
date_tag=20231223
cd /cvmfs/neurodesk.ardc.edu.au/containers/deepretinotopy_1.0.2_"$date_tag"/deepretinotopy_1.0.2_$date_tag.simg/opt/deepRetinotopy_TheToolbox/
bash deepRetinotopy.sh -s /home/jovyan/data/ -t /home/jovyan/templates/ -d hcp -m "polarAngle"

tee: deepRetinotopy_output.log: Read-only file system


Path subs directory: /home/jovyan/data/
Path to fs_LR-deformed_to-fsaverage surfaces: /home/jovyan/templates/
Dataset name: hcp
Maps: polarAngle
Generating mid-thickness surface and curvature data...


tee: deepRetinotopy_error.log: Read-only file system


Path: /home/jovyan/data/
Hemisphere: lh
Subject: 102311
Generating mid-thickness surface and curvature data if not available...
Preparing native surfaces for resampling if not available...
Execution time: 0 minutes
Retinotopy prediction...


/opt/miniconda-py37_4.8.3/lib/python3.7/site-packages/torch_geometric/nn/conv/spline_conv.py:118: UserWarning: We do not recommend using the non-optimized CPU version of `SplineConv`. If possible, please move your data to GPU.
  'We do not recommend using the non-optimized CPU version of '


Loading the dataset
Processing...
Done!
Dataset loaded
Time elapsed: 0.5105423666536808 minutes
Loading model 1
Saving .gii files in: /home/jovyan/data/102311/deepRetinotopy/
Predictions from model 1 were saved
Time elapsed: 5.247529104351997 minutes
Loading model 2
Saving .gii files in: /home/jovyan/data/102311/deepRetinotopy/
Predictions from model 2 were saved
Time elapsed: 5.431513167917728 minutes
Loading model 3
Saving .gii files in: /home/jovyan/data/102311/deepRetinotopy/
Predictions from model 3 were saved
Time elapsed: 5.80756638199091 minutes
Loading model 4
Saving .gii files in: /home/jovyan/data/102311/deepRetinotopy/
Predictions from model 4 were saved
Time elapsed: 5.306253969669342 minutes
Loading model 5
Saving .gii files in: /home/jovyan/data/102311/deepRetinotopy/
Predictions from model 5 were saved
Time elapsed: 5.170940432697535 minutes
Saving average predictions
Average predictions were saved
Resampling predictions to native space...
Path: /home/jovyan/data/
Hemis

/opt/miniconda-py37_4.8.3/lib/python3.7/site-packages/torch_geometric/nn/conv/spline_conv.py:118: UserWarning: We do not recommend using the non-optimized CPU version of `SplineConv`. If possible, please move your data to GPU.
  'We do not recommend using the non-optimized CPU version of '


Resampling predictions to native space...
Path: /home/jovyan/data/
Hemisphere: rh
Map: polarAngle
Model: model1
Resampling fsaverage to native space...
Path: /home/jovyan/data/
Hemisphere: rh
Map: polarAngle
Model: model2
Resampling fsaverage to native space...
Path: /home/jovyan/data/
Hemisphere: rh
Map: polarAngle
Model: model3
Resampling fsaverage to native space...
Path: /home/jovyan/data/
Hemisphere: rh
Map: polarAngle
Model: model4
Resampling fsaverage to native space...
Path: /home/jovyan/data/
Hemisphere: rh
Map: polarAngle
Model: model5
Resampling fsaverage to native space...
Path: /home/jovyan/data/
Hemisphere: rh
Map: polarAngle
Model: average
Resampling fsaverage to native space...
polarAngle predictions for rh are done!
All done!


## Visualization

In [1]:
# imports
import numpy as np
import pandas as pd
import random
import sys

sys.path.append('.')

from functions.visualization import polarAngle_plot
from ipywidgets import interact, Dropdown

In [5]:
subject_id=str(102311)
prediction = Dropdown(options = ['average','model1', 'model2', 'model3', 'model4', 'model5'])
binarize = Dropdown(options = [False, True])

@interact(prediction = prediction, binarize = binarize)
def plot1(prediction, binarize):
    return polarAngle_plot(subject_id, '/home/jovyan/data', '/home/jovyan/templates', prediction = prediction, binarize = binarize)

interactive(children=(Dropdown(description='prediction', options=('average', 'model1', 'model2', 'model3', 'mo…